In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms

In [2]:
import sys
sys.path.insert(1, "../../")

In [3]:
from data_preprocessing import get_means, get_stds, ImageNetSubset
from Models.darknet19 import Darknet19

In [4]:
if torch.cuda.is_available():
    device=torch.device('cuda')
else:
    device=torch.device('cpu') 

In [5]:
data_path = '../../../dummy_datasets/'
norms_path = os.path.join(data_path, 'norms.json')

In [6]:
means = get_means(path=norms_path, train_loader=None)
stds = get_stds(path=norms_path, train_loader=None)

Means are: [0.4405549168586731, 0.4407285749912262, 0.4381718039512634]
stds are: [0.25142669677734375, 0.25270089507102966, 0.25131651759147644]


In [7]:
transformations = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize(mean=means, std=stds)])

In [8]:
train_dataset = ImageNetSubset(path=data_path, train=True, transform=transformations, half=False, show=False)

# Inference

In [9]:
model = Darknet19(num_classes=1000, device=device, dtype=torch.float32)

In [10]:
img = train_dataset[0][0].unsqueeze(0)

In [11]:
from torch.profiler import profile, ProfilerActivity

In [158]:
with profile(activities=[ProfilerActivity.CPU],
                                    schedule=torch.profiler.schedule(wait=1, warmup=2, active=10, repeat=1),
                                    on_trace_ready=torch.profiler.tensorboard_trace_handler('../log/darknet19/inference'),
                                    record_shapes=True,
                                    profile_memory=True,
                                    with_flops=True,
                                    with_modules=True,
                                    with_stack=True) as prof:
    for i in range(13):
        out = model(img)
        prof.step()
print(prof.key_averages().table(row_limit=-1))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg       CPU Mem  Self CPU Mem    # of Calls  Total GFLOPs  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                    ProfilerStep*         1.87%      13.470ms       100.00%     719.479ms      71.948ms      60.63 Mb    -545.76 Mb            10            --  
                       Conv block         6.61%      47.561ms        82.21%     591.478ms       3.286ms     517.35 Mb           0 b           180            --  
                     aten::conv2d         0.21%       1.499ms        63.68%     458.174ms       2.411ms     174.13 Mb           0 b           190        55.820  
                aten::convol